## Week 4 Exercise - AI Code Assistant

This project demonstrates an integrated **AI Code Assistant**. It transitions beyond simple chat interfaces to a specialized tool designed to automate the most repetitive and critical parts of the Software Development Life Cycle (SDLC). 

### Core Capabilities
* **Generate Docstring:** Generates standardized, high-quality documentation (Google/NumPy style) for raw functions to improve codebase maintainability.
* **Generate Unit Tests:** Automatically synthesizes a robust suite of unit tests using `pytest` or language-equivalent frameworks, focusing on logic coverage and edge cases.
* **Translate Code:** A sophisticated cross-compiler that translates logic between languages (e.g., Java to Python, JS to Go) while maintaining idiomatic syntax.
* **Optimize & Improve:** Analyzes code for algorithmic bottlenecks and refactors it for better time/space complexity and readability.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display
load_dotenv(override=True)

In [ ]:

OPEN_ROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPEN_ROUTER_API_KEY = os.getenv('OPEN_ROUTER_API_KEY')

client = OpenAI(
    base_url=OPEN_ROUTER_BASE_URL,
    api_key=OPEN_ROUTER_API_KEY
)

MODELS = [
    "openai/gpt-4.1-mini",
    "anthropic/claude-sonnet-4.6",
    "google/gemini-3-flash-preview"
]

TASKS = [
    "Generate Docstring",
    "Generate Unit Tests",
    "Translate Code",
    "Optimize & Improve"
]

LANGS = ["Python", "JavaScript", "Java", "C#", "C++", "Go", "Rust", "TypeScript", "PHP"]

PYTHON_CODE = """def calculate_total(prices, tax_rate):
    total = sum(prices)
    return total + (total * tax_rate)"""

JAVA_CODE = """public class Greeter {
    public static void main(String[] args) {
        System.out.println("Hello, World!");
    }
}"""

JAVASCRIPT_CODE = """function findDuplicates(items) {
    const duplicates = [];
    for (let i = 0; i < items.length; i++) {
        for (let j = i + 1; j < items.length; j++) {
            if (items[i] === items[j] && !duplicates.includes(items[i])) {
                duplicates.push(items[i]);
            }
        }
    }
    return duplicates;
}
"""

In [ ]:
def process_code(code_input, task, model, source_lang, target_lang):
    instructions = {
        TASKS[0]: "Add clear Google-style docstrings and comments.",
        TASKS[1]: "Generate a suite of unit tests using a popular framework for this language.",
        TASKS[2]: f"Translate this code from {source_lang} to {target_lang}. Ensure idiomatic syntax.",
        TASKS[3]: "Refactor this code for better performance and readability."
    }
    
    
    system_message = f"You are a senior developer. {instructions[task]}"
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": code_input}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
result = process_code(
    code_input=JAVASCRIPT_CODE,
    task=TASKS[1],
    model=MODELS[0],
    source_lang="JavaScript",
    target_lang="Python"
)
display(Markdown(result))

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("# AI Code Assistant")
    
    with gr.Row():
        with gr.Column():
            code_in = gr.Code(label="Source Code", lines=15, value=PYTHON_CODE)
            
            with gr.Row():
                task_dropdown = gr.Dropdown(
                    choices=TASKS, 
                    label="Task",
                    value=TASKS[0]
                )
                model_radio = gr.Radio(
                    choices=MODELS, 
                    label="Model", 
                    value=MODELS[0]
                )
            
            with gr.Row(visible=False) as lang_row:
                src_lang = gr.Dropdown(choices=LANGS, label="From", value="Java")
                tgt_lang = gr.Dropdown(choices=LANGS, label="To", value="Python")

            run_btn = gr.Button("Execute Task", variant="primary")
            
            gr.Examples(
                examples=[
                    [PYTHON_CODE, TASKS[0], "Python", "Java"],
                    [JAVA_CODE, TASKS[1], "Java", "Python"],
                    [JAVASCRIPT_CODE, TASKS[2], "JavaScript", "Python"]
                ],
                inputs=[code_in, task_dropdown, src_lang, tgt_lang],
                label="Click an example to test the toolkit"
            )
            
        with gr.Column():
            code_out = gr.Markdown(label="Result")

    def toggle_langs(task):
        if task == TASKS[2]:
            return gr.update(visible=True)
        return gr.update(visible=False)

    task_dropdown.change(fn=toggle_langs, inputs=task_dropdown, outputs=lang_row)

    run_btn.click(
        fn=process_code, 
        inputs=[code_in, task_dropdown, model_radio, src_lang, tgt_lang], 
        outputs=code_out
    )

ui.launch()